# TFDS and Rock, Paper, Scissors

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%203%20-%20TensorFlow%20Datasets/Week%201/Exercises/TFDS_Week1_Exercise.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%203%20-%20TensorFlow%20Datasets/Week%201/Exercises/TFDS_Week1_Exercise.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

In this week's exercise you will be working with TFDS and the rock-paper-scissors dataset. You'll do a few tasks such as exploring the info of the dataset in order to figure out the name of the splits. You'll also write code to see if the dataset supports the new S3 API before creating your own versions of the dataset.

## Setup

In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

In [2]:
# Use all imports
import tensorflow as tf
import tensorflow_datasets as tfds

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.0.0


## Extract the Rock, Paper, Scissors Dataset

In the cell below, you will extract the `rock_paper_scissors` dataset and then print its info. Take note of the splits, what they're called, and their size.

In [14]:
# EXERCISE: Use tfds.load to extract the rock_paper_scissors dataset.

data, info = tfds.load(name="rock_paper_scissors", split=('train', 'test'), data_dir='/tmp', with_info=True)# YOUR CODE HERE
print(info)

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    version=3.0.0,
    description='Images of hands playing rock, paper, scissor game.',
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=2892,
    splits={
        'test': 372,
        'train': 2520,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
    redistribution_info=,
)



In [20]:
data

(<DatasetV1Adapter shapes: {image: (300, 300, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (300, 300, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>)

In [22]:
info.splits

{'test': <tfds.core.SplitInfo num_examples=372>,
 'train': <tfds.core.SplitInfo num_examples=2520>}

In [38]:
# EXERCISE: In the space below, write code that iterates through the splits
# without hardcoding any keys. The code should extract 'test' and 'train' as
# the keys, and then print out the number of items in the dataset for each key. 
# HINT: num_examples property is very useful here.

for k in info.splits:# YOUR CODE HERE:
  print(k + ":" + str(info.splits[k].num_examples)) # YOUR CODE HERE)

# EXPECTED OUTPUT
# test:372
# train:2520

test:372
train:2520


## Use the New S3 API

Before using the new S3 API, you must first find out whether the `rock_paper_scissors` dataset implements the new S3 API. In the cell below you should use version `3.*.*` of the `rock_paper_scissors` dataset.

In [39]:
# EXERCISE: In the space below, use the tfds.builder to fetch the
# rock_paper_scissors dataset and check to see if it supports the
# new S3 API. 
# HINT: The builder should 'implement' something

rps_builder = tfds.builder('rock_paper_scissors:3.*.*')# YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")

print(rps_builder.version.implements(tfds.core.Experiment.S3)) # YOUR CODE HERE)

# Expected output:
# True

True


## Create New Datasets with the S3 API

Sometimes datasets are too big for prototyping. In the cell below, you will create a smaller dataset, where instead of using all of the training data and all of the test data, you instead have a `small_train` and `small_test` each of which are comprised of the first 10% of the records in their respective datasets.

In [40]:
# EXERCISE: In the space below, create two small datasets, `small_train`
# and `small_test`, each of which are comprised of the first 10% of the
# records in their respective datasets.

small_train = tfds.load('rock_paper_scissors:3.*.*',  split='train[:10%]') # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")
small_test = tfds.load('rock_paper_scissors:3.*.*', split='test[:10%]') # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")

# No expected output yet, that's in the next cell

/root/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/root/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteRC6KU2/rock_paper_scissors-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteRC6KU2/rock_paper_scissors-test.tfrecord


Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [41]:
# EXERCISE: Print out the size (length) of the small versions of the datasets.

print(len(list(small_train))) # YOUR CODE HERE)
print(len(list(small_test))) # YOUR CODE HERE)

# Expected output
# 252
# 37

252
37


The original dataset doesn't have a validation set, just training and testing sets. In the cell below, you will use TFDS to create new datasets according to these rules:

* `new_train`: The new training set should be the first 90% of the original training set.


* `new_test`: The new test set should be the first 90% of the original test set.


* `validation`: The new validation set should be the last 10% of the original training set + the last 10% of the original test set.

In [43]:
# EXERCISE: In the space below, create 3 new datasets according to
# the rules indicated above.

new_train = tfds.load('rock_paper_scissors:3.*.*', split='train[:90%]') # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")
print(len(list(new_train))) # YOUR CODE HERE)

new_test = tfds.load('rock_paper_scissors:3.*.*', split='test[:90%]') # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")
print(len(list(new_test))) # YOUR CODE HERE)

validation = tfds.load('rock_paper_scissors:3.*.*', split='train[:-90%] + test[:-90%]') # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")
print(len(list(validation)))# YOUR CODE HERE)

# Expected output
# 2268
# 335
# 289

2268
335
289
